# Store Sales - Time Series Forecasting

### Use machine learning to predict grocery sales

Kaggle link: [Store Sales - Time Series Forecasting](https://www.kaggle.com/competitions/store-sales-time-series-forecasting)

### Context
Forecasts aren’t just for meteorologists. Governments forecast economic growth. Scientists attempt to predict the future population. And businesses forecast product demand—a common task of professional data scientists. Forecasts are especially relevant to brick-and-mortar grocery stores, which must dance delicately with how much inventory to buy. Predict a little over, and grocers are stuck with overstocked, perishable goods. Guess a little under, and popular items quickly sell out, leading to lost revenue and upset customers. More accurate forecasting, thanks to machine learning, could help ensure retailers please customers by having just enough of the right products at the right time.

Current subjective forecasting methods for retail have little data to back them up and are unlikely to be automated. The problem becomes even more complex as retailers add new locations with unique needs, new products, ever-transitioning seasonal tastes, and unpredictable product marketing.

### Potential Impact
If successful, you'll have flexed some new skills in a real world example. For grocery stores, more accurate forecasting can decrease food waste related to overstocking and improve customer satisfaction. The results of this ongoing competition, over time, might even ensure your local store has exactly what you need the next time you shop.

### Dataset Description
In this competition, you will predict sales for the thousands of product families sold at Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

### File Descriptions and Data Field Information

**train.csv**
The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
**store_nbr** identifies the store at which the products are sold.
**family** identifies the type of product sold.
**sales** gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
**onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date.

**test.csv**
The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
The dates in the test data are for the 15 days after the last date in the training data.

**sample_submission.csv**
A sample submission file in the correct format.

**stores.csv**
Store metadata, including city, state, type, and cluster.
cluster is a grouping of similar stores.

**oil.csv**
Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

**holidays_events.csv**
* Holidays and Events, with metadata
* NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
* Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

### Additional Notes

* Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
* A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

### Importing all libraries

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.linear_model import *
from math import *
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.feature_extraction import *
from sklearn.naive_bayes import *
from sklearn.discriminant_analysis import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from sklearn.neighbors import *
from sklearn.cluster import *
from sklearn.svm import *
import warnings
warnings.filterwarnings("ignore")
from googletrans import *

### Importing the training datasets

In [34]:
df_train_1 = pd.read_csv("train1.csv")
df_train_1.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,1/1/13,1,AUTOMOTIVE,0.0,0
1,1,1/1/13,1,BABY CARE,0.0,0
2,2,1/1/13,1,BEAUTY,0.0,0
3,3,1/1/13,1,BEVERAGES,0.0,0
4,4,1/1/13,1,BOOKS,0.0,0


In [35]:
df_train_2 = pd.read_csv("train2.csv")
df_train_2.head()

,id,date,store_nbr,family,sales,onpromotion
0,262143,5/28/13,14,MEATS,310.173000,0
1,262144,5/28/13,14,PERSONAL CARE,227.000000,0
2,262145,5/28/13,14,PET SUPPLIES,0.000000,0
3,262146,5/28/13,14,PLAYERS AND ELECTRONICS,0.000000,0
4,262147,5/28/13,14,POULTRY,47.809002,0


In [36]:
df_train_3 = pd.read_csv("train3.csv")
df_train_3.head()

,id,date,store_nbr,family,sales,onpromotion
0,524286,10/22/13,2,HOME AND KITCHEN I,0.0,0
1,524287,10/22/13,2,HOME AND KITCHEN II,0.0,0
2,524288,10/22/13,2,HOME APPLIANCES,0.0,0
3,524289,10/22/13,2,HOME CARE,0.0,0
4,524290,10/22/13,2,LADIESWEAR,0.0,0


In [37]:
df_train_4 = pd.read_csv("train4.csv")
df_train_4.head()

,id,date,store_nbr,family,sales,onpromotion
0,786429,3/19/14,25,CELEBRATION,8.000,0
1,786430,3/19/14,25,CLEANING,485.000,0
2,786431,3/19/14,25,DAIRY,588.000,0
3,786432,3/19/14,25,DELI,116.357,0
4,786433,3/19/14,25,EGGS,119.000,0


In [38]:
print("Total length of training dataframes : ",(len(df_train_1)+len(df_train_2)+len(df_train_3)+len(df_train_4)))

Total length of training dataframes :  1048575


### Merging all training dataframes into one single dataframe

In [39]:
a = df_train_1.append(df_train_2)
b = df_train_3.append(df_train_4)
df_train = a.append(b)
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,1/1/13,1,AUTOMOTIVE,0.0,0
1,1,1/1/13,1,BABY CARE,0.0,0
2,2,1/1/13,1,BEAUTY,0.0,0
3,3,1/1/13,1,BEVERAGES,0.0,0
4,4,1/1/13,1,BOOKS,0.0,0


In [40]:
print("Total length of merged dataframe : ",(len(df_train)))

Total length of merged dataframe :  1048575


### Importing testing dataset

In [41]:
df_test = pd.read_csv("test.csv")
df_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [42]:
print("Total length of testing dataframe : ",(len(df_test)))

Total length of testing dataframe :  28512


### Checking for null values in both training and testing datasets

In [43]:
df_train.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [44]:
df_test.isna().sum()

id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64

### Exploring other datasets

In [45]:
df_stores = pd.read_csv("stores.csv")
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [46]:
df_oil = pd.read_csv("oil.csv")
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [47]:
df_holiday_events = pd.read_csv("holidays_events.csv")
df_holiday_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [48]:
df_transactions = pd.read_csv("transactions.csv")
df_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [49]:
print("Total length of stores dataframe         :  ",len(df_stores))
print("Total length of oil dataframe            :  ",len(df_oil))
print("Total length of holiday events dataframe :  ",len(df_holiday_events))
print("Total length of transactions dataframe   :  ",len(df_transactions))

Total length of stores dataframe         :   54
Total length of oil dataframe            :   1218
Total length of holiday events dataframe :   350
Total length of transactions dataframe   :   83488


### Checking for null values in stores, oil, holiday events, transactions dataframes

In [50]:
df_stores.isna().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [51]:
df_oil.isna().sum()

date           0
dcoilwtico    43
dtype: int64

In [52]:
df_holiday_events.isna().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [53]:
df_transactions.isna().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

### Filling in null values in oil dataset

In [54]:
df_oil_nona = df_oil.copy()
modal = df_oil_nona["dcoilwtico"].mode()
na = df_oil_nona[df_oil_nona["dcoilwtico"].isna()==True].index.values
df_oil_nona.loc[0,"dcoilwtico"] = df_oil_nona.loc[0:10,"dcoilwtico"].mean()
for i in na[1:]:
    df_oil_nona.loc[i,"dcoilwtico"] = df_oil_nona.loc[(i-10):(i+10),"dcoilwtico"].mean()
df_oil_nona.head()

,date,dcoilwtico
0,2013-01-01,93.366
1,2013-01-02,93.140
2,2013-01-03,92.970
3,2013-01-04,93.120
4,2013-01-07,93.200


In [55]:
df_oil_nona.isna().sum()

date          0
dcoilwtico    0
dtype: int64

In [56]:
df_holiday_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


### Preprocessing the holiday events dataset

In [57]:
before_transfer = df_holiday_events[df_holiday_events["transferred"]==True]
before_transfer

,date,type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
255,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
266,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
268,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
297,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
303,2017-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,True
312,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
324,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


In [58]:
after_transfer = df_holiday_events[df_holiday_events["type"]=="Transfer"]
after_transfer

,date,type,locale,locale_name,description,transferred
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
73,2013-10-11,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
136,2014-10-10,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
256,2016-05-27,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
265,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False
269,2016-08-12,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
298,2017-01-02,Transfer,National,Ecuador,Traslado Primer dia del ano,False
304,2017-04-13,Transfer,Local,Cuenca,Fundacion de Cuenca,False
313,2017-05-26,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
325,2017-08-11,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False


In [64]:
tr = Translator()
tr.translate("안녕하세요")

AttributeError: 'NoneType' object has no attribute 'group'

In [63]:
help(Translator)

Help on class Translator in module googletrans.client:

class Translator(builtins.object)
 |  Translator(service_urls=None, user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64)', proxies=None, timeout=None)
 |  
 |  Google Translate ajax API implementation class
 |  
 |  You have to create an instance of Translator to use this API
 |  
 |  :param service_urls: google translate url list. URLs will be used randomly.
 |                       For example ``['translate.google.com', 'translate.google.co.kr']``
 |  :type service_urls: a sequence of strings
 |  
 |  :param user_agent: the User-Agent header to send when making requests.
 |  :type user_agent: :class:`str`
 |  
 |  :param proxies: proxies configuration. 
 |                  Dictionary mapping protocol or protocol and host to the URL of the proxy 
 |                  For example ``{'http': 'foo.bar:3128', 'http://host.name': 'foo.bar:4012'}``
 |  :type proxies: dictionary
 |  
 |  :param timeout: Definition of timeout for Request

In [73]:
df_holiday_events[df_holiday_events["type"]=="Bridge"]

,date,type,locale,locale_name,description,transferred
35,2012-12-24,Bridge,National,Ecuador,Puente Navidad,False
39,2012-12-31,Bridge,National,Ecuador,Puente Primer dia del ano,False
156,2014-12-26,Bridge,National,Ecuador,Puente Navidad,False
160,2015-01-02,Bridge,National,Ecuador,Puente Primer dia del ano,False
277,2016-11-04,Bridge,National,Ecuador,Puente Dia de Difuntos,False


In [78]:
df_holiday_events["type"].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [80]:
df_holiday_events[df_holiday_events["type"]=="Additional"]

,date,type,locale,locale_name,description,transferred
28,2012-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False
33,2012-12-22,Additional,National,Ecuador,Navidad-3,False
34,2012-12-23,Additional,National,Ecuador,Navidad-2,False
36,2012-12-24,Additional,National,Ecuador,Navidad-1,False
38,2012-12-26,Additional,National,Ecuador,Navidad+1,False
40,2012-12-31,Additional,National,Ecuador,Primer dia del ano-1,False
53,2013-05-11,Additional,National,Ecuador,Dia de la Madre-1,False
64,2013-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False
81,2013-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
